In [27]:
import os
import json
import astropy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import Distance
from astropy.cosmology import Planck15

In [58]:
def absmag(m, z):
    '''
    Converts apparent magnitude to absolute magnitude.
    
    Args:
        m (float): apparent magnitude
        z (float): redshift
    '''
    if m is None or m is np.nan or z is None or z is np.nan:
        return np.nan
    else:
        if z==0:
            M = m
        else:
            d = Distance(unit=u.pc, z=float(z), allow_negative=True)
            M = m - 5 * np.log10(d/(10*u.pc)) + 5
        return M
absmag = np.vectorize(absmag)

In [2]:
df_ztfrest = pd.read_csv('ZTFReST.csv')
ztfredshifts = df_ztfrest.dropna(subset=["redshift"]).copy().reset_index(drop=True)

objIds = sorted(list(set(ztfredshifts["id"])))

In [3]:
data_folder = 'data/'
photometry_dict = {}


for root, dirs, files in sorted(os.walk(data_folder)):
    for file in files:
        if file == 'photometry.json':
            folder_name = os.path.basename(root)
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                photometry_dict[folder_name] = json.load(f)

In [4]:
lc_dict = {key: pd.DataFrame.from_dict(val) for key, val in photometry_dict.items()}

In [5]:
# Initialize an empty dictionary to store non-empty items
flc_dict = {}
elc_dict = {}

# Iterate over the items in lc_dict
for key, value in lc_dict.items():
    # Check if the value is not empty
    if not value.empty:
        # If the value is not empty, add it to the new dictionary
        flc_dict[key] = value
    else:
        # If the value is empty, print the key
        print(f"{key}")
        elc_dict[key] = value

# Now non_empty_lc_dict contains only the items from lc_dict that are not empty

print('number of non-empty lightcurves:', len(flc_dict))
print('number of empty lightcurves:', len(elc_dict))

194.94985-47.75
Swift-J002325.5-141701
Swift-J181727.0-192921
ZTF18abfcmjwm
ZTF18abianhw
ZTF18abukavnm
ZTF18abvkwlam
ZTF19aankdan
ZTF19aapuudk
ZTF19aasexmy
ZTF19abeyvoi
ZTF19abobxik
ZTF19abrpfps
ZTF19abuvqgw
ZTF19acaxbjt
ZTF19acayojs
ZTF19accxzsc
ZTF20aaelulum
ZTF20aazrcbp
ZTF20ababxjv
ZTF20abmocba
ZTF20acigmelm
ZTF22abvvmti
number of non-empty lightcurves: 190
number of empty lightcurves: 23


In [18]:
df_eval = pd.read_csv('candidateEval06-17.csv')
display(df_eval.head())

## print all objects that have an uncertainty in the redshift, then strip the uncertainty from the redshift column
print('objects with uncertainties in redshift:')
for index, row in df_eval.iterrows():
    if '±' in str(row['redshift']):
        print(row['object'])
        df_eval.loc[index, 'redshift'] = float(row['redshift'].split('±')[0])
        
## print number of objects with redshifts
print('number of objects with redshifts:', len(df_eval[df_eval['redshift'].notnull()]))
print('number of objects without redshifts:', len(df_eval[df_eval['redshift'].isnull()]))
print('number of objects with redshifts and classifications:', len(df_eval[df_eval['redshift'].notnull() & df_eval['class'].notnull()]))
print('unique classifications:')
[print(i) for i in df_eval['class'].unique() if pd.notnull(i)];

,object,redshift,class,captures_ztfg_rise,captures_ztfg_fade,captures_ztfr_rise,captures_ztfr_fade,has_ztfi,ztfg_rise,ztfg_fade,ztfr_rise,ztfr_fade,ztfi_rise,ztfi_fade,usability,notes,citation
0,ZTF18aakuewf,0.0636,Ibn,n,y,y,y,n,NaN,58225-58240,58225-58230,58230-58250,NaN,NaN,4,NaN,https://www.aanda.org/articles/aa/abs/2021/05/...
1,ZTF18aalrxas,0.0582 ± 0.0003,IIb,y,y,n,y,n,58227-58230,58229-58235,NaN,58229-58232,NaN,NaN,3,lowish quality ztfr,https://arxiv.org/abs/1903.09262
2,ZTF18abantmh,NaN,NaN,n,n,n,n,n,NaN,NaN,NaN,NaN,NaN,NaN,0,low quality,NaN
3,ZTF18abffyqp,0.031,II,m,m,m,m,n,58307-58309,58308-58315,58307-58309,58308-58315,NaN,NaN,3,data gap at peak; odd second peak,https://arxiv.org/abs/2008.09986
4,ZTF18abvkmgw,0.03847,Ib,y,y,y,y,n,58373-58378,58377-58383,58375-58378,58377-58386,NaN,NaN,3,decent g,https://www.wis-tns.org/object/2018ghd


objects with uncertainties in redshift:
ZTF18aalrxas
ZTF20abummyz
ZTF22acaruqr
number of objects with redshifts: 79
number of objects without redshifts: 111
number of objects with redshifts and classifications: 72
unique classifications:
Ibn
IIb
II
Ib
SN
long GRB
Ia
GRB
Novae
orphan
unclear
afterglow
FOT
FOT;orphan
U Gem
Cataclysmic
Ib/c;Ib-pec
Tidal Disruption Event
Novae;Classical Nova
FBOT
FBOT; Cataclysmic
afterglow; Ic-BL
IIP


In [20]:
hq_cands = df_eval[df_eval['usability'] >= 4]
mq_cands = df_eval[(df_eval['usability'] < 4) & (df_eval['usability'] > 2)]
lq_cands = df_eval[(df_eval['usability'] < 3) & (df_eval['usability'] >= 1)]
z_cands = df_eval.dropna(subset=['redshift'])
hflc_dict = {key: value for key, value in flc_dict.items() if key in hq_cands['object'].values} ## high quality
mflc_dict = {key: value for key, value in flc_dict.items() if key in mq_cands['object'].values} ## medium quality
lflc_dict = {key: value for key, value in flc_dict.items() if key in lq_cands['object'].values} ## low quality
zlc_dict =  {key: value for key, value in flc_dict.items() if key in z_cands['object'].values} ## redshift 

In [59]:
display(zlc_dict['ZTF18aakuewf'].head())
for item in zlc_dict:
    zlc_dict[item]['absmag'] = absmag(zlc_dict[item]['mag'], z_cands[z_cands['object'] == item]['redshift'].values[0])

,obj_id,ra,dec,filter,mjd,snr,instrument_id,instrument_name,ra_unc,dec_unc,...,id,altdata,created_at,groups,annotations,mag,magerr,magsys,limiting_mag,absmag
0,ZTF18aakuewf,NaN,NaN,ztfr,58197.500532,NaN,1,ZTF,None,None,...,201111494,None,2022-04-12T21:52:29.589147,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],NaN,NaN,ab,18.992901,NaN
1,ZTF18aakuewf,NaN,NaN,ztfr,58198.394132,NaN,1,ZTF,None,None,...,201112158,None,2022-04-12T21:52:32.025953,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],NaN,NaN,ab,20.430599,NaN
2,ZTF18aakuewf,NaN,NaN,ztfr,58198.398623,NaN,1,ZTF,None,None,...,201112159,None,2022-04-12T21:52:32.026048,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],NaN,NaN,ab,20.457500,NaN
3,ZTF18aakuewf,NaN,NaN,ztfr,58198.435081,NaN,1,ZTF,None,None,...,201112160,None,2022-04-12T21:52:32.026070,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],NaN,NaN,ab,20.568701,NaN
4,ZTF18aakuewf,NaN,NaN,ztfr,58198.455567,NaN,1,ZTF,None,None,...,201112161,None,2022-04-12T21:52:32.026087,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],NaN,NaN,ab,20.582800,NaN


/home/tbarna/anaconda3/envs/nmma_env/lib/python3.10/site-packages/astropy/units/quantity.py:666: RuntimeWarning: divide by zero encountered in log10
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/tbarna/anaconda3/envs/nmma_env/lib/python3.10/site-packages/astropy/units/quantity.py:666: RuntimeWarning: invalid value encountered in log10
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [55]:
display(zlc_dict['ZTF18aakuewf'][zlc_dict['ZTF18aakuewf']['absmag'].notnull()])

,obj_id,ra,dec,filter,mjd,snr,instrument_id,instrument_name,ra_unc,dec_unc,...,id,altdata,created_at,groups,annotations,mag,magerr,magsys,limiting_mag,absmag
23,ZTF18aakuewf,243.594398,35.917906,ztfr,58226.340532,34.222286,1,ZTF,None,None,...,201112176,None,2022-04-12T21:52:32.026310,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],18.961000,0.031726,ab,20.611401,-13.389765
24,ZTF18aakuewf,243.594397,35.917906,ztfr,58226.358623,36.912226,1,ZTF,None,None,...,201112177,None,2022-04-12T21:52:32.026324,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],18.871901,0.029414,ab,20.676800,-13.478865
26,ZTF18aakuewf,243.594392,35.917891,ztfr,58226.378947,32.416809,1,ZTF,None,None,...,201112179,None,2022-04-12T21:52:32.026353,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],18.922800,0.033493,ab,20.597799,-13.427966
28,ZTF18aakuewf,243.594377,35.917912,ztfr,58226.414456,34.338093,1,ZTF,None,None,...,201111499,None,2022-04-12T21:52:29.589311,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],18.856501,0.031619,ab,20.611500,-13.494265
30,ZTF18aakuewf,243.594403,35.917928,ztfr,58226.415394,32.527524,1,ZTF,None,None,...,201112181,None,2022-04-12T21:52:32.026381,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],18.849501,0.033379,ab,20.528200,-13.501265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3965,ZTF18aakuewf,243.594426,35.917928,ztfg,60224.139502,5.530807,1,ZTF,None,None,...,546074056,None,2024-03-21T23:15:38.060879,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],19.689800,0.196307,ab,20.078899,-12.660965
3969,ZTF18aakuewf,243.594382,35.917963,ztfg,60231.125058,5.871650,1,ZTF,None,None,...,546074058,None,2024-03-21T23:15:38.060911,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],19.708668,0.184912,ab,20.072325,-12.642098
3972,ZTF18aakuewf,243.594441,35.917938,ztfg,60351.434097,5.709683,1,ZTF,None,None,...,546074059,None,2024-03-21T23:15:38.060926,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],19.691099,0.190157,ab,20.349300,-12.659666
3981,ZTF18aakuewf,243.594399,35.917935,ztfg,60355.514213,5.302430,1,ZTF,None,None,...,546074063,None,2024-03-21T23:15:38.060987,"[{'id': 13, 'created_at': '2020-10-21T06:20:33...",[],19.732599,0.204762,ab,20.308100,-12.618166


In [13]:
# fig, axs = plt.subplots(10,7, figsize=(20, 20))
# color = {'ztfg': 'g', 'ztfr': 'r', 'ztfi': 'b'}
# for i, (key, value) in enumerate(hflc_dict.items()):
#     ax = axs[i//7, i%7]
#     min_time = value[(value['mag'].notnull())][(value['filter'] == 'ztfg') | (value['filter'] == 'ztfr') | (value['filter'] == 'ztfi')]['mjd'].min()
#     ax.set_xlim(min_time-2, min_time+25)
#     ax.invert_yaxis(), ax.grid()
#     ax.set_title(f'{key}')
#     # ax.set_xlabel('MJD')
#     # ax.set_ylabel('Magnitude')
#     for filt in ['ztfg', 'ztfr', 'ztfi']:
#         filt_df = value[(value['filter'] == filt)  & (value['mag'].notnull())]
#         if filt_df.empty:
#             continue
#         else:
#             ax.errorbar(filt_df['mjd'], filt_df['mag'], yerr=filt_df['magerr'], fmt='o', label=filt, color=color[filt])
# fig.tight_layout()
# plt.show()

In [14]:
# fig, axs = plt.subplots(10,7, figsize=(20, 20))
# color = {'ztfg': 'g', 'ztfr': 'r', 'ztfi': 'b'}
# fast_fade_count = 0
# for i, (key, value) in enumerate(hflc_dict.items()):
#     ax = axs[i//7, i%7]
#     min_time = value[(value['mag'].notnull())][(value['filter'] == 'ztfg') | (value['filter'] == 'ztfr') | (value['filter'] == 'ztfi')]['mjd'].min()
#     ax.set_xlim(min_time-2, min_time+25)
#     ax.invert_yaxis(), ax.grid()
#     ax.set_title(f'{key}')
#     # ax.set_xlabel('MJD')
#     # ax.set_ylabel('Magnitude')
#     rise_rates = {}
#     fade_rates = {}
#     for filt in ['ztfg', 'ztfr', 'ztfi']:
#         filt_df = value[(value['filter'] == filt)  & (value['mag'].notnull())]
#         if filt_df.empty:
#             continue
#         else:
#             ax.errorbar(filt_df['mjd'], filt_df['mag'], yerr=filt_df['magerr'], fmt='o', label=filt, color=color[filt])
        
#         ## find if it has rise or fade values in the candidateEval05-30.csv file
#         try:
#             rise_time_range = hq_cands[hq_cands['Object'] == key][f'{filt} rise min/max'].values[0]
#             if type(rise_time_range) == str:
#                 rise_time_range = rise_time_range.split('-')
#                 rise_time_range = [float(rise_time_range[0]), float(rise_time_range[1])]
#                 first_data_point = filt_df[filt_df['mjd'] >= rise_time_range[0]].iloc[0]
                
#                 last_data_point = filt_df[filt_df['mjd'] <= rise_time_range[1]].iloc[-1]
#                 rise_rate = (last_data_point['mag'] - first_data_point['mag'])/(last_data_point['mjd'] - first_data_point['mjd'])
#                 rise_rates.update({filt: round(rise_rate,2)})
#                 # print(rise_rate)
#                 # ax.text(0.05, 0.95, f'Rise rate: {rise_rate:.2f}', transform=ax.transAxes, fontsize=8, verticalalignment='top')
#         except:
#             print('issue with rise rate for ', key, filt)
#             pass
#         try:
#             fade_time_range = hq_cands[hq_cands['Object'] == key][f'{filt} fade min/max'].values[0]
#             if type(fade_time_range) == str:
#                 fade_time_range = fade_time_range.split('-')
#                 fade_time_range = [float(fade_time_range[0]), float(fade_time_range[1])]
#                 first_data_point = filt_df[filt_df['mjd'] >= fade_time_range[0]].iloc[0]
#                 last_data_point = filt_df[filt_df['mjd'] <= fade_time_range[1]].iloc[-1]
#                 fade_rate = (last_data_point['mag'] - first_data_point['mag'])/(last_data_point['mjd'] - first_data_point['mjd'])
#                 fade_rates.update({filt: round(fade_rate,2)})
#                 # print(fade_rate)
#                 # ax.text(0.05, 0.95, f'Fade rate: {fade_rate:.2f}', transform=ax.transAxes, fontsize=8, verticalalignment='top')
#                 # if fade_rate > 0.3:
#                 #     print('{} has a fade rate above 0.3 in {}'.format(key, filt))
#         except:
#             print('issue with fade rate for ', key, filt)
#             pass
#     ## format dictionary to string for plotting
#     if any([float(r) > 0.3 for r in fade_rates.values()]):
#         fast_fade_count += 1
#     rise_rates = ', '.join([f'{k}: {v}' for k,v in rise_rates.items()])
#     fade_rates = ', '.join([f'{k}: {v}' for k,v in fade_rates.items()])
#     ax.text(0.05, 0.20, f'Rise: {rise_rates}', transform=ax.transAxes, fontsize=8, verticalalignment='top') if rise_rates else None
#     ax.text(0.05, 0.10, f'Fade: {fade_rates}', transform=ax.transAxes, fontsize=8, verticalalignment='top') if fade_rates else None
    
# print('number of lightcurves with fade rates above 0.3:', fast_fade_count)       
# fig.tight_layout()
# plt.show()